## Lab 11: Window Functions - grouping, but better!

Let's start by reviewing something we already know how to do - grouping based on some value set and then calculating things like the average or mean. 

We learnt how to do this using group_by() in lab 10 (before our data viz work!) - let's revise quickly. 

In [1]:
import pandas as pd

In [3]:
# let's play with our old toy dataset for now
data = pd.DataFrame(
    data=[
        ['312', 'A1', 0.12, 'LEFT'],
        ['312', 'A2', 0.37, 'LEFT'],
        ['312', 'C2', 0.68, 'LEFT'],
        ['313', 'A1', 0.07, 'RIGHT'],
        ['313', 'B1', 0.08, 'RIGHT'],
        ['314', 'A2', 0.29, 'LEFT'],
        ['314', 'B1', 0.14, 'RIGHT'],
        ['314', 'C2', 0.73, 'RIGHT'],
        ['711', 'A1', 4.01, 'RIGHT'],
        ['712', 'A2', 3.29, 'LEFT'],
        ['713', 'B1', 5.74, 'LEFT'],
        ['714', 'B2', 3.32, 'RIGHT'],
    ],
    columns=['subject_id', 'condition_id', 'response_time', 'response'],
)
data

,subject_id,condition_id,response_time,response
0,312,A1,0.12,LEFT
1,312,A2,0.37,LEFT
2,312,C2,0.68,LEFT
3,313,A1,0.07,RIGHT
4,313,B1,0.08,RIGHT
5,314,A2,0.29,LEFT
6,314,B1,0.14,RIGHT
7,314,C2,0.73,RIGHT
8,711,A1,4.01,RIGHT
9,712,A2,3.29,LEFT


In [5]:
data.groupby('condition_id')['response_time'].mean()

condition_id
A1    1.400000
A2    1.316667
B1    1.986667
B2    3.320000
C2    0.705000
Name: response_time, dtype: float64

In [7]:
data.groupby('subject_id')['response_time'].max()

subject_id
312    0.68
313    0.08
314    0.73
711    4.01
712    3.29
713    5.74
714    3.32
Name: response_time, dtype: float64

What happens to the output - how does the size and shape change? What if we don't want our output to be a different shape or size? 

Or what if we want to group by something that isn't a categorical variable? What if we want to aggregate across a particualr time window? Or number of rows in a sliding fashion? 

E.g. what if we wanted to calculate the sum of every 5 rows at a time? 

Of course, we could do this with a for loop, but what about a more efficient approach (for our big health datasets!)? Time for "windows"!

## Rolling Windows

Imagine you are a public health researcher with a dataset which spans many months and years and tracks some metrics of health across different groups or regions (sound familiar? most of your project datasets will likely look a bit like this). 

Now imagine you want to calculate some values of these metrics as they change over time. How can we do that? 

For example, let's say if your data is about the number of COVID-19 infections recorded in a  clinic. Any day there were positive cases the clinic logged them, if there were no cases then nothing was logged. You want to aggregate over different period of time, maybe over different windows of time to understand the average infection rates in a more dynamic way to grasp the disease dynmaics themselves.  

Let's build a toy version of that dataset and see can we figure out how to do that using "window" functions. 

In [9]:
times = ['2021-01-01', '2021-01-03', '2021-01-04', '2021-01-05', '2021-01-29'] # let's set up some dates

s = pd.Series(range(5), index=pd.DatetimeIndex(times)) # and add an integer to each date that will represent our "recorded positive cases". 
s

2021-01-01    0
2021-01-03    1
2021-01-04    2
2021-01-05    3
2021-01-29    4
dtype: int64

Okay, let's say we want to start summing over some number of these rows to build a more "big picture" view of these numbers. Remember the rolling weekly infection rates from 2020?

Jump here for a quick look: https://www.nyc.gov/site/doh/covid/covid-19-data.page#daily 

Let's try and get some of those kinds of numbers... 


The rolling() function will let us do just that by specifying a window that will sum the number of rows specified as our "window". 
Let's say we want to start simple, summing up a rolling addition every two days...

In [11]:
# Window with 2 observations
s.rolling(window=2).sum()


2021-01-01    NaN
2021-01-03    1.0
2021-01-04    3.0
2021-01-05    5.0
2021-01-29    7.0
dtype: float64

What do we notice? 

What happened with the shape of our dataframe? How is this different to something like group_by()?
What about the first row, what happened there? 

But is this actually summing over a certain number of days? Look more carefully. 
Our days are evenly separated, some have gaps and some skip many days so we aren't actually getting a "two days rolling sum" like we hoped. 

Let's get more specific with our window function - it is really good at handling date items and understands units of time...

In [13]:
# Window with 2 days worth of observations
s.rolling(window='2D').sum()

2021-01-01    0.0
2021-01-03    1.0
2021-01-04    3.0
2021-01-05    5.0
2021-01-29    4.0
dtype: float64

Note that the unit of time here is days, it cannot be months as months have different numbers of days, if you tried something like "1M" you would get a  window error (try it and see what the error message is!)

In [15]:
s.rolling(window='1M').sum() # this will throw an error - examine it as part of improving your debugging!

C:\Users\sa4422\AppData\Local\Temp\ipykernel_39324\3550118969.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  s.rolling(window='1M').sum() # this will throw an error - examine it as part of improving your debugging!


ValueError: <MonthEnd> is a non-fixed frequency

If you want to center the calculation around the current row then set center = True, spot the difference in these outputs to understand the difference. 

In [17]:
s #these are the start value

2021-01-01    0
2021-01-03    1
2021-01-04    2
2021-01-05    3
2021-01-29    4
dtype: int64

In [19]:
s.rolling(window=3).sum()

2021-01-01    NaN
2021-01-03    NaN
2021-01-04    3.0
2021-01-05    6.0
2021-01-29    9.0
dtype: float64

In [21]:
s.rolling(window=3, center = True).sum()

2021-01-01    NaN
2021-01-03    3.0
2021-01-04    6.0
2021-01-05    9.0
2021-01-29    NaN
dtype: float64

## Expanding windows

Okay - what if we want to watch positive cases accumulating over time - with each new row being added to the count? Again we could write a for loop - but that's rarely the best option!

For this, we have the expanding() function which is used for cumulative or expanding window calculations. Unlike rolling(), which applies operations over a fixed-size window, expanding() grows the window size as it moves through the data. It starts at the first element and includes all prior elements up to the current one.

In [23]:
s # remember what we have already... 

2021-01-01    0
2021-01-03    1
2021-01-04    2
2021-01-05    3
2021-01-29    4
dtype: int64

In [25]:
s.expanding(min_periods=1).sum()

2021-01-01     0.0
2021-01-03     1.0
2021-01-04     3.0
2021-01-05     6.0
2021-01-29    10.0
dtype: float64

In [27]:
s.expanding(min_periods=1).mean()

2021-01-01    0.0
2021-01-03    0.5
2021-01-04    1.0
2021-01-05    1.5
2021-01-29    2.0
dtype: float64

Note - these expanding calculations are a special case of rolling statistics. We could achieve an equivalent output with the following rolling() call...

In [29]:
s.rolling(window=len(s), min_periods=1).mean()

2021-01-01    0.0
2021-01-03    0.5
2021-01-04    1.0
2021-01-05    1.5
2021-01-29    2.0
dtype: float64

Why does this happen what is rolling() capturing with each step to achieve this?

## Shifting rows

The .shift() function is used to shift the values of a column or index by a specified number of periods, either forward or backward. It's useful for creating lagged or lead features, comparing data points across different time steps (also good for time series analysis...)

Basic use case - you can use .shift() to calculate differences between current and past values.

In [31]:
data = {'values': [10, 20, 30, 40, 50]}
df = pd.DataFrame(data)
df

,values
0,10
1,20
2,30
3,40
4,50


In [33]:
# Shifting values down by 1 period (default)
df['shifted_down'] = df['values'].shift(1)
df

,values,shifted_down
0,10,NaN
1,20,10.0
2,30,20.0
3,40,30.0
4,50,40.0


In [35]:
# Shifting values up by 1 period
df['shifted_up'] = df['values'].shift(-1)
df

,values,shifted_down,shifted_up
0,10,NaN,20.0
1,20,10.0,30.0
2,30,20.0,40.0
3,40,30.0,50.0
4,50,40.0,NaN


In [37]:
# Create a new column for the difference between the current and previous row
df['diff'] = df['values'] - df['values'].shift(1)
df

,values,shifted_down,shifted_up,diff
0,10,NaN,20.0,NaN
1,20,10.0,30.0,10.0
2,30,20.0,40.0,10.0
3,40,30.0,50.0,10.0
4,50,40.0,NaN,10.0


## Exercise/Challenge: Back to our neural data 

#For each patcher, compute the average number of days they waited between experiments

Here is how to proceed
1. Use a window function to compute the number of days that elapse between experiment (i.e., the distance between `date`), for each `patcher`. Add that as a new column, `'days from prev'`
2. Compute the average `'days from prev'` per patcher

With your new awesome vectorization skills, it should only take two lines! (though you may have to lay around first to get here!

In [39]:
# Set some Pandas options: maximum number of rows/columns it's going to display
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)

In [41]:
df = pd.read_csv('experiment_data.csv', parse_dates=['date']) # load in the data

In [43]:
df.head()

,OP,patcher,date,slice,cell_ch,cell_ID,day,treatment,hrs_incubation,repatch,hrs_after_OP,Rs,Rin,resting_potential,max_spikes,Rheobase,AP_heigth,TH,max_depol,max_repol,membra_time_constant_tau,capacitance,comments,rheo_ramp,AP_halfwidth,Rheobse_ramp,Unnamed: 27,rheos_ramp,comment,,high K concentration,RMP_from_char,tissue_source,area,patient_age
0,OP211209,Verji,2024-03-13,S2,8,21d10S2c8,D1,Ctrl,0.0,no,13.298889,14.470281,166.878916,-67.962646,34,50.0,83.190918,-36.132812,302.124023,-72.631836,20.75,152.623120,NaN,NaN,0.966102,NaN,NaN,NaN,NaN,NaN,15 mM,-59.101382,Bielefeld,temporal,27.0
1,OP221024,Verji,2024-06-16,S3,4,22o24S3c4,D1,Ctrl,0.0,no,23.964167,11.521243,137.820797,-71.789551,41,50.0,93.322754,-42.968750,465.820312,-83.740234,14.85,124.324170,17,NaN,0.959995,NaN,NaN,NaN,NaN,NaN,8 mM,-62.265689,Bielefeld,temporal,42.0
2,OP230810,Verji,2024-05-14,S2,5,23810S2c5,D1,TTX,0.0,no,7.043056,10.120637,67.739416,-70.629883,47,100.0,91.973877,-37.817383,415.771484,-107.666016,13.00,228.654858,10,402.013400,0.760052,NaN,NaN,NaN,NaN,NaN,8 mM,-61.329228,Mitte,temporal,63.0
3,OP230209,Verji,2024-04-27,S2_D2,3,23209S2_D2c3,D2,high K,25.0,no,21.848333,7.745503,43.009610,-68.371582,31,500.0,67.163086,-29.284668,212.036133,-61.645508,11.05,215.784505,30,672.202407,0.958735,NaN,NaN,NaN,NaN,NaN,8 mM,-62.577472,Bielefeld,temporal,63.0
4,OP240321,Verji,2024-04-11,S2,4,24321S2c4,D1,Ctrl,0.0,no,11.530278,7.632941,32.884808,-52.453613,21,200.0,84.008789,-36.785889,403.442383,-71.899414,14.80,695.791105,8,NaN,1.063838,324.520817,NaN,NaN,NaN,NaN,8 mM,-63.149769,Bielefeld,temporal,31.0


In [45]:
df['patcher'].unique() # we have two none NAN values here .... 

array(['Verji', 'Rosie', nan], dtype=object)

In [47]:
df['days from prev'] = df['date'] - df.sort_values('date').groupby('patcher')['date'].shift()
df

,OP,patcher,date,slice,cell_ch,cell_ID,day,treatment,hrs_incubation,repatch,hrs_after_OP,Rs,Rin,resting_potential,max_spikes,Rheobase,AP_heigth,TH,max_depol,max_repol,membra_time_constant_tau,capacitance,comments,rheo_ramp,AP_halfwidth,Rheobse_ramp,Unnamed: 27,rheos_ramp,comment,,high K concentration,RMP_from_char,tissue_source,area,patient_age,days from prev
0,OP211209,Verji,2024-03-13,S2,8,21d10S2c8,D1,Ctrl,0.00,no,13.298889,14.470281,166.878916,-67.962646,34,50.0,83.190918,-36.132812,302.124023,-72.631836,20.75,152.623120,NaN,NaN,0.966102,NaN,NaN,NaN,NaN,NaN,15 mM,-59.101382,Bielefeld,temporal,27.0,0 days
1,OP221024,Verji,2024-06-16,S3,4,22o24S3c4,D1,Ctrl,0.00,no,23.964167,11.521243,137.820797,-71.789551,41,50.0,93.322754,-42.968750,465.820312,-83.740234,14.85,124.324170,17,NaN,0.959995,NaN,NaN,NaN,NaN,NaN,8 mM,-62.265689,Bielefeld,temporal,42.0,0 days
2,OP230810,Verji,2024-05-14,S2,5,23810S2c5,D1,TTX,0.00,no,7.043056,10.120637,67.739416,-70.629883,47,100.0,91.973877,-37.817383,415.771484,-107.666016,13.00,228.654858,10,402.013400,0.760052,NaN,NaN,NaN,NaN,NaN,8 mM,-61.329228,Mitte,temporal,63.0,1 days
3,OP230209,Verji,2024-04-27,S2_D2,3,23209S2_D2c3,D2,high K,25.00,no,21.848333,7.745503,43.009610,-68.371582,31,500.0,67.163086,-29.284668,212.036133,-61.645508,11.05,215.784505,30,672.202407,0.958735,NaN,NaN,NaN,NaN,NaN,8 mM,-62.577472,Bielefeld,temporal,63.0,0 days
4,OP240321,Verji,2024-04-11,S2,4,24321S2c4,D1,Ctrl,0.00,no,11.530278,7.632941,32.884808,-52.453613,21,200.0,84.008789,-36.785889,403.442383,-71.899414,14.80,695.791105,8,NaN,1.063838,324.520817,NaN,NaN,NaN,NaN,8 mM,-63.149769,Bielefeld,temporal,31.0,2 days
5,OP220127,Rosie,2024-07-22,S2_D2,5,22129S2_D2c5,D2,TTX,24.00,no,46.794444,9.807291,53.633033,-66.906738,30,150.0,77.740479,-35.937500,347.656250,-69.824219,11.40,227.361656,31,NaN,0.923105,NaN,NaN,NaN,NaN,31.0,15 mM,-61.293551,Mitte,temporal,45.0,2 days
6,OP221024,Verji,2024-06-14,S2_D2,4,22o24S2c5,D2,TTX,24.00,yes,47.464167,9.592057,38.531748,-63.372803,6,600.0,60.595703,-22.961426,158.203125,-41.381836,3.95,117.773976,30,NaN,1.239090,NaN,NaN,NaN,NaN,NaN,8 mM,-63.762181,Bielefeld,temporal,42.0,1 days
7,OP221027,Verji,2024-02-01,S2_D2,2,22o27S2c2,D2,TTX,24.00,yes,34.658889,13.086921,53.729798,-69.470215,33,200.0,76.171875,-34.063721,290.283203,-88.989258,10.90,276.234493,30,NaN,0.806014,NaN,NaN,NaN,NaN,NaN,8 mM,-61.882100,Mitte,temporal,61.0,1 days
8,OP240215,Verji,2024-02-10,S1,4,24215S1c4,D1,Ctrl,0.00,no,11.725833,10.029166,45.268596,-74.066162,25,150.0,88.140869,-42.742920,333.007812,-79.956055,19.25,486.340786,3,NaN,0.958975,719.513984,NaN,NaN,NaN,NaN,8 mM,-60.029570,Bielefeld,temporal,30.0,1 days
9,OP230808,Verji,2024-04-04,S5,4,23808S5c4,D1,Ctrl,0.00,no,24.792778,8.488248,52.494468,-74.884033,23,250.0,88.897705,-37.536621,437.011719,-64.208984,15.95,493.531256,28,513.557119,1.143393,NaN,NaN,NaN,NaN,NaN,8 mM,-65.061362,Hamburg,temporal,14.0,1 days


In [49]:
df.sort_values(['patcher', 'date'])[['patcher', 'date', 'days from prev']].head() # this is just to view your data more intuitively

,patcher,date,days from prev
251,Rosie,2024-01-01,NaT
102,Rosie,2024-01-04,3 days
355,Rosie,2024-01-04,0 days
47,Rosie,2024-01-05,1 days
477,Rosie,2024-01-05,0 days


In [51]:
df.groupby('patcher')['days from prev'].mean() # this is to calculate those means

patcher
Rosie   1 days 05:45:15.789473684
Verji   0 days 10:53:42.269807280
Name: days from prev, dtype: timedelta64[ns]